In [196]:
import pandas as pd
import numpy as np

datapath="../data/"

submission_data = pd.read_csv(datapath + "submission.csv")
test_data = pd.read_csv(datapath + "test.csv")
train_data = pd.read_csv(datapath + "train.csv")
countrycodes=pd.read_csv(datapath+"country-codes_csv.csv")
indicator_data=pd.read_excel(datapath+"Indicator_list.xlsx").fillna("")

indicator_data["long_form"]=indicator_data["long_form"].apply(lambda x: "_".join(x.split(" ")))
train_data[["ConfirmedCases","Fatalities"]]=train_data[["ConfirmedCases","Fatalities"]].astype(int)
submission_data[["ConfirmedCases","Fatalities"]]=submission_data[["ConfirmedCases","Fatalities"]].astype(int)

In [22]:
train_data

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0,0
1,2,NaN,Afghanistan,2020-01-23,0,0
2,3,NaN,Afghanistan,2020-01-24,0,0
3,4,NaN,Afghanistan,2020-01-25,0,0
4,5,NaN,Afghanistan,2020-01-26,0,0
...,...,...,...,...,...,...
19105,29361,NaN,Zimbabwe,2020-03-22,3,0
19106,29362,NaN,Zimbabwe,2020-03-23,3,1
19107,29363,NaN,Zimbabwe,2020-03-24,3,1
19108,29364,NaN,Zimbabwe,2020-03-25,3,1


In [247]:
import requests
from tqdm import tqdm

def import_data(indicator="",indicator_type="",name=""):
    url="https://ghoapi.azureedge.net/api/{}".format(indicator)
    data = pd.DataFrame(requests.get(url).json()["value"])
    data.drop([col for col in data.columns if data[col].isna().sum() == data.shape[0]],axis=1,inplace=True)
    data=data.drop(data[data.NumericValue.isna()].index,axis=0).reset_index(drop=True)
    data=data[data.Dim1==indicator_type] if len(indicator_type)>0 else data
    data=data[["IndicatorCode","SpatialDim","TimeDim","NumericValue"]].drop_duplicates(subset=["IndicatorCode","SpatialDim"],keep="last").reset_index(drop=True)
    data.columns=["IndicatorCode","Shortcut","Year",name]
    
    return data

def import_who_indicators(indicator_data):
    data=[import_data(indicator["short_form"],indicator["addition"],indicator["long_form"]) for _,indicator in tqdm(indicator_data.iterrows())]
    return data
    
who_data_list=import_who_indicators(indicator_data)

11it [00:11,  1.06s/it]


In [190]:
def match_country_code(countrycodes,traindata):
    countrycodes=countrycodes[["ISO3166-1-Alpha-3","official_name_en"]]
    countrycodes=countrycodes.drop(countrycodes[countrycodes["ISO3166-1-Alpha-3"].isna()].index,axis=0).reset_index(drop=True)
    countrycodes.columns = ["Shortcut", "Country_Region"]
    result = traindata.merge(countrycodes,on="Country_Region",how="left").reset_index(drop=True)
    return result

new_traindata = match_country_code(countrycodes,train_data)

def match_dataframes(new_traindata,who_data):
    result = new_traindata.merge(who_data,on="Shortcut",how="left")
    return result
match_dataframes(new_traindata,who_data)

(19110, 7)


,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Shortcut,IndicatorCode,Year,HealthServiceProvision
0,1,NaN,Afghanistan,2020-01-22,0,0,AFG,IHRSPAR_CAPACITY09,2018.0,60.0
1,2,NaN,Afghanistan,2020-01-23,0,0,AFG,IHRSPAR_CAPACITY09,2018.0,60.0
2,3,NaN,Afghanistan,2020-01-24,0,0,AFG,IHRSPAR_CAPACITY09,2018.0,60.0
3,4,NaN,Afghanistan,2020-01-25,0,0,AFG,IHRSPAR_CAPACITY09,2018.0,60.0
4,5,NaN,Afghanistan,2020-01-26,0,0,AFG,IHRSPAR_CAPACITY09,2018.0,60.0
...,...,...,...,...,...,...,...,...,...,...
19105,29361,NaN,Zimbabwe,2020-03-22,3,0,ZWE,IHRSPAR_CAPACITY09,2018.0,20.0
19106,29362,NaN,Zimbabwe,2020-03-23,3,1,ZWE,IHRSPAR_CAPACITY09,2018.0,20.0
19107,29363,NaN,Zimbabwe,2020-03-24,3,1,ZWE,IHRSPAR_CAPACITY09,2018.0,20.0
19108,29364,NaN,Zimbabwe,2020-03-25,3,1,ZWE,IHRSPAR_CAPACITY09,2018.0,20.0


In [248]:
[data.Year for data in who_data_list]

[0      2017
 1      2017
 2      2017
 3      2017
 4      2017
        ... 
 189    2017
 190    2017
 191    2017
 192    2017
 193    2017
 Name: Year, Length: 194, dtype: int64, 0      2011
 1      2015
 2      2012
 3      2010
 4      2006
        ... 
 175    2007
 176    2014
 177    2010
 178    2008
 179    2006
 Name: Year, Length: 180, dtype: int64, 0      2014
 1      2014
 2      2014
 3      2014
 4      2014
        ... 
 198    2014
 199    2014
 200    2014
 201    2014
 202    2014
 Name: Year, Length: 203, dtype: int64, 0      2001
 1      1997
 2      2010
 3      2003
 4      2000
        ... 
 188    1999
 189    1997
 190    2004
 191    2004
 192    1990
 Name: Year, Length: 193, dtype: int64, 0      2000
 1      2005
 2      2005
 3      2001
 4      2008
        ... 
 187    2006
 188    1999
 189    2017
 190    2015
 191    2017
 Name: Year, Length: 192, dtype: int64, 0      2007
 1      2009
 2      2016
 3      2012
 4      2012
        ... 
 180    2014

In [249]:
list(set(who_data_list[6].Year))

[2025, 2012]